<a href="https://colab.research.google.com/github/lonerpatti/BigDataPos/blob/main/Uni_Facef_Pyspark_Atividade_Nota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'>Uni-Facef - Pyspark - Atividade</font>

Essa atividade tem como objetivo descobrir um pouco mais sobre o cliente e traçar um perfil do mesmo. Visto que temos as informações pessoais do cliemte em um arquivo chamado "cliente_fake.csv", e os dados transacionais desse cliente no arquivo chamado "venda_fake". E idéia do exercicio é a manipulação dos dados de forma a fazer agregações, join nos dados, podendo ser necessário o uso de UDFs, gerando um Dataframe Final:

#### O Dataframe final deve estar agrupado por cliente (cliente_id), e conter as seguintes informações:

- cliente_id - O identificador do cliente
- idade - Calcular a idade do cliente (valor inteiro)
- recencia - Calcular a quantos meses faz que o cliente não compra (valor inteiro)
- qt_total_compras - Quantidade total de compras (pedidos distintos)
- vr_total_compras = Valor total de compras do cliente
- qt_total_itens - Quantidade total de itens comprados pelo cliente (produto_id distintos)
- qt_max_parcelas - Quantidade máxima de parcelas que o cliente já utilizou
- perfil_cliente - Deve mostrar o perfil de compra desse cliente, se compra somente "loja" ou somente "online", ou em ambos "multicanal";

OBS: O dataset "venda_fake" está a nível de itens. Pode haver mais de um item por pedido, ito é, o numero do pedido pode estar duplicado nodataset. 


In [2]:
from google.colab import files
uploaded = files.upload()

Saving cliente_fake.csv to cliente_fake.csv
Saving venda_fake.csv to venda_fake.csv


In [3]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 65kB/s 
     |████████████████████████████████| 204kB 42.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=99161db1e7d1dcec85d1ba8bacf666185914bc3dafe14604a79bb50ba610cb7a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [5]:
# coding: utf-8
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
    .appName('ConhecendoCliente') \
    .getOrCreate()

#### Dataset de cliente

In [7]:
df_cliente = spark.read \
    .option("delimiter", "|") \
    .csv('cliente_fake.csv', header=True)

df_cliente.show()

+--------------+----------+-------------+
|          nome|cliente_id|dt_nascimento|
+--------------+----------+-------------+
|Cliente 100013|    100013|   1964-06-21|
|Cliente 100014|    100014|   1964-05-24|
|Cliente 100015|    100015|   1946-10-18|
|Cliente 100016|    100016|   1972-03-24|
|Cliente 100017|    100017|   1982-01-19|
|Cliente 100018|    100018|   1983-04-26|
|Cliente 100019|    100019|   1971-09-09|
|Cliente 100020|    100020|   1989-06-18|
|Cliente 100007|    100007|   1976-11-04|
|Cliente 100008|    100008|   1980-12-01|
|Cliente 100009|    100009|   1981-03-05|
|Cliente 100010|    100010|   1984-03-20|
|Cliente 100011|    100011|   1977-11-22|
|Cliente 100012|    100012|   1974-02-04|
|Cliente 100001|    100001|   2009-04-16|
|Cliente 100002|    100002|   1974-08-29|
|Cliente 100003|    100003|   1981-12-05|
|Cliente 100004|    100004|   1968-08-08|
|Cliente 100005|    100005|   1982-11-29|
|Cliente 100006|    100006|   1948-04-01|
+--------------+----------+-------

#### Dataset de pedido

In [8]:
df_venda = spark.read \
    .option("delimiter", "|") \
    .csv('venda_fake.csv', header=True)

df_venda.show()

+--------------+----------+--------------------+----------+-----------+--------------+------------+-------+--------+
|     pedido_id|cliente_id|           dt_pedido|produto_id|qt_parcelas|   forma_pagto|vr_unit_item|qt_item|tp_canal|
+--------------+----------+--------------------+----------+-----------+--------------+------------+-------+--------+
|10000720200217|    100007|2020-02-17T00:00:...|   6253644|         20|cartao credito|         478|      1|    loja|
|10000720200217|    100007|2020-02-17T00:00:...|   5313408|         20|cartao credito|      1357.2|      1|    loja|
|10000720200217|    100007|2020-02-17T00:00:...|   6253140|         20|cartao credito|         372|      1|    loja|
|10000720200625|    100007|2020-06-25T00:00:...|   6858780|          7|cartao credito|       317.8|      1|    site|
|10000120200626|    100001|2020-06-26T00:00:...|   7560984|         16|cartao credito|     1399.04|      1|    loja|
|10000920200614|    100009|2020-06-14T00:00:...|   7571280|     